In [1]:
import java.nio.file.Files
import java.nio.file.Path

data class BenchmarkData(val name: String, val score: Double, val error: Double) {
    val type = getType(name)
    override fun toString(): String = "$name: $score ± $error ns/op"
}

enum class Type { Int, Long, Float, Double }

fun getType(name: String) = when {
    name.contains("Float") -> Type.Float
    name.contains("Int") -> Type.Int
    name.contains("Double") -> Type.Double
    name.contains("Long") -> Type.Long // Long must be the last as it is contained in other test names
    else -> error("Unknown type: ${name}")
}

fun readHotspotBenchmarkData(benchmarkName: String) = Files.readAllLines(Path.of("build", "results", "jmh", "results.txt"))
    .mapNotNull {
        val r = Regex("$benchmarkName\\.([^ ]+) +avgt +\\d+ +([0-9.]+) +± +([0-9.]+) +ns/op")
        val match = r.matchEntire(it) ?: return@mapNotNull null
        BenchmarkData(
            name = match.groups[1]!!.value,
            score = match.groups[2]!!.value.toDouble(),
            error = match.groups[3]!!.value.toDouble(),
        )
    }.sortedBy { it.type }
(readHotspotBenchmarkData("Wolf3dBenchmark") + readHotspotBenchmarkData("FibonacciBenchmark")).joinToString("\n")

baselineFloat: 3.2691445983E7 ± 69466.905 ns/op
inlineFloat: 3.242113406E7 ± 70299.166 ns/op
longPackFloat: 5.6830523627E7 ± 241134.675 ns/op
mutableRefFloatIn1Long: 6.3871743398E7 ± 572515.259 ns/op
mutableRefFloatIn2Longs: 3.8639579995E7 ± 46378.366 ns/op
valueFloat: 2.4868718136E7 ± 47922.097 ns/op
valueInlineFloat: 2.492274722E7 ± 42271.19 ns/op
valuePreserveBoxFloat: 2.4950436818E7 ± 41974.954 ns/op
baselineDouble: 3.368255698E7 ± 124420.029 ns/op
inlineDouble: 3.4301550173E7 ± 135677.939 ns/op
mutableRefDouble: 3.9446891111E7 ± 169020.613 ns/op
valueDouble: 2.4467271516E7 ± 31685.302 ns/op
valueInlineDouble: 2.4547361612E7 ± 162281.24 ns/op
valuePreserveBoxDouble: 2.4487710636E7 ± 28913.444 ns/op

In [2]:
%use lets-plot

In [3]:
import org.jetbrains.letsPlot.intern.PosKind
import org.jetbrains.letsPlot.intern.StatKind
import org.jetbrains.letsPlot.intern.layer.PosOptions
import org.jetbrains.letsPlot.intern.layer.StatOptions

fun makePlot(name: String, platform: String, benchmarkLines: List<BenchmarkData>) {
    if (benchmarkLines.isEmpty()) return
    val names = benchmarkLines.map { it.name }
    val scores = benchmarkLines.map { it.score }
    val scoresMinusErrors = benchmarkLines.map { it.score - it.error / 2 }
    val scoresPlusErrors = benchmarkLines.map { it.score + it.error / 2 }
    val baseline = benchmarkLines.filter { it.name.startsWith("baseline") }.groupBy { it.type }.mapValues { (_, v) -> v.single() }
    val maxScore = scoresPlusErrors.max()
    val plot = letsPlot() +
            labs("$name ($platform)", fill = "Benchmark", y = "ns/op") +
            geomBar(stat = StatOptions(kind = StatKind.IDENTITY)) {
                x = names
                y = scores
                fill = names
            } +
            geomErrorBar(
                width = 0.5,
                position = PosOptions(kind = PosKind.DODGE),
                size = 0.6
            ) {
                x = names
                ymin = scoresMinusErrors
                ymax = scoresPlusErrors
            } +
            geomText(
                stat = StatOptions(kind = StatKind.IDENTITY),
                position = positionStack(vjust = 1.0),
                size = 4.5,
            ) {
                x = names
                y = scores.map {
                    val threshold = 0.08
                    if (it < maxScore * threshold) it + threshold * maxScore / 2 else it / 2
                }
                label = benchmarkLines.map {
                    val score = it.score / baseline[it.type]!!.score
                    val error = it.error / baseline[it.type]!!.score
                    "${String.format("%.0f", score * 100)}%\n±\n${String.format("%.1f", error * 100)}%\n${it.type}"
                }
            } +
            scaleXDiscrete(
                labels = names,
                name = "",
                limits = names + listOf(""),
                breaks = names.toList()
            ) +
            scaleYContinuous(limits = 0 to maxScore, name = "ns/op") +
            theme(plotTitle = elementText(hjust = 0.5)).legendPositionRight() +
            ggsize(1000, 800)
    ggsave(plot = plot, filename = "$name ($platform).svg")
    plot.show()
}


In [4]:
makePlot("Wolf3d", "Hotspot", readHotspotBenchmarkData("Wolf3dBenchmark"))
makePlot("Ackermann", "Hotspot", readHotspotBenchmarkData("AckermannBenchmark"))
makePlot("BoxRecreation", "Hotspot", readHotspotBenchmarkData("BoxRecreationBenchmark"))

In [5]:
import java.io.FileFilter
import kotlin.io.path.listDirectoryEntries

fun readArtBenchmarkData(benchmarkName: String) = Path.of("android-benchmark/build/outputs/androidTest-results/connected")
    .toFile().listFiles(FileFilter { it.isDirectory })!!.single()
    .listFiles { dir, name -> name.startsWith("logcat-org.jetbrains.") }!!.flatMap { it.readLines() }
    .mapNotNull {
        val pattern = Regex("^\\d{2}-\\d{2} +\\d+:\\d+:\\d+.\\d+ +\\d+ +\\d+ +I +Benchmark: +$benchmarkName\\.([^\\[]+)\\[Metric \\(timeNs\\) +results: +median +([^,]+), +min [^,]+, +max [^,]+, +standardDeviation: +([^,]+), +.*")
        val match = pattern.matchEntire(it) ?: return@mapNotNull null
        BenchmarkData(
            name = match.groups[1]!!.value,
            score = match.groups[2]!!.value.toDouble(),
            error = match.groups[3]!!.value.toDouble(),
        )
     }.let { lines ->
         val indexes = readHotspotBenchmarkData(benchmarkName).mapIndexed { index, benchmarkData -> benchmarkData.name to index }.toMap()
         if (lines.all { it.name in indexes }) lines.sortedBy { indexes[it.name]!! } else lines.sortedBy { it.type }
     }
(readArtBenchmarkData("Wolf3dBenchmark") + readArtBenchmarkData("FibonacciBenchmark")).joinToString("\n")

baselineFloat: 3.08391077E8 ± 6947896.167355742 ns/op
inlineFloat: 8.36105788E8 ± 1.3108268500106793E7 ns/op
longPackFloat: 7.77419805E7 ± 1624176.4022136945 ns/op
mutableRefFloatIn1Long: 1.595877495E8 ± 2635529.401398846 ns/op
mutableRefFloatIn2Longs: 1.01489596E8 ± 1955132.4707032915 ns/op
valueFloat: 3.118959615E8 ± 8983614.184004676 ns/op
valueInlineFloat: 1.86374308E7 ± 35957.79321975998 ns/op
valuePreserveBoxFloat: 3.603156535E8 ± 1.1222725355574882E7 ns/op
baselineDouble: 4.695191535E8 ± 1.1491546939985685E7 ns/op
inlineDouble: 1.0423727495E9 ± 8.119720564459203E7 ns/op
mutableRefDouble: 1.06578058E8 ± 1481233.2264560792 ns/op
valueDouble: 5.60696615E8 ± 1.1736808642243443E7 ns/op
valueInlineDouble: 2.5887717833333332E7 ± 29790.72557306751 ns/op
valuePreserveBoxDouble: 5.341226535E8 ± 1.2601148601575267E7 ns/op

In [6]:
makePlot("Wolf3d", "ART", readArtBenchmarkData("Wolf3dBenchmark"))
makePlot("Ackermann", "ART", readArtBenchmarkData("AckermannBenchmark"))
makePlot("BoxRecreation", "ART", readArtBenchmarkData("BoxRecreationBenchmark"))